In [134]:
import psycopg2
import pandas as pd
from bokeh.plotting import figure, show, output_file

In [135]:
#  HEROKU - SAVE

# see next block
# http://blog.y3xz.com/blog/2012/08/16/flask-and-postgresql-on-heroku
# app.config['SQLALCHEMY_DATABASE_URI'] = os.environ['DATABASE_URL']
# db = SQLAlchemy(app)


# conn = psycopg2.connect(
#     database=pawel,
#     user=url.username,
#     password=url.password,
#     host=url.hostname,
#     port=url.port
# )

In [0]:
# http://stackoverflow.com/questions/31473457/converting-query-results-into-dataframe-in-python
# QUOTES ARE important: """ vs ' vs "
# http://www.postgresql.org/docs/current/static/sql-syntax-lexical.html#SQL-SYNTAX-IDENTIFIERS

In [194]:

indicators = [('AG.LND.TOTL.K2', 'Land area (sq. km)'),
              ('EN.POP.DNST', 'Population density (people per sq. km of land area)'),
              ('ER.BDV.TOTL.XQ', 'GEF benefits index for biodiversity (0 = no biodiversity potential to 100 = maximum)'),
              ('VC.PKP.TOTL.UN', 'Presence of peace keepers (number of troops, police, and military observers in mandate)')]
#ind1 = app.vars['i1']
# ind2 = app.vars['i2']

ind1 = 'AG.LND.TOTL.K2'
ind2 = 'IS.AIR.PSGR'
ind = (ind1, ind2)
print ind
repr(ind)

"('AG.LND.TOTL.K2', 'IS.AIR.PSGR')"

('AG.LND.TOTL.K2', 'IS.AIR.PSGR')


In [198]:
conn = psycopg2.connect(database="wb_indicators")
#query = """SELECT "Country", "Indicator", "zscore" from usn WHERE "Indicator" IN ('AG.LND.TOTL.K2', 'SI.POV.GINI');"""
query = """SELECT country, indicator, zscore from usn WHERE indicator IN (%s, %s);"""
print query
cursor = conn.cursor()
cursor.execute(query, (ind1, ind2))
df0=pd.DataFrame(cursor.fetchall(), columns=['country', 'indicator', 'zscore'])

# conn.commit();
# conn.close();

df = df0.pivot('country', 'indicator')
df = df.zscore

print df0.head()
print df.head()

SELECT country, indicator, zscore from usn WHERE indicator IN (%s, %s);
     country       indicator    zscore
0    Algeria  AG.LND.TOTL.K2  0.261822
1    Algeria     IS.AIR.PSGR -0.403979
2  Argentina  AG.LND.TOTL.K2  0.377283
3  Argentina     IS.AIR.PSGR -0.356186
4  Australia  AG.LND.TOTL.K2  1.986030
indicator   AG.LND.TOTL.K2  IS.AIR.PSGR
country                                
Algeria           0.261822    -0.403979
Argentina         0.377283    -0.356186
Australia         1.986030     0.163806
Austria          -0.486082    -0.309165
Azerbaijan       -0.486040    -0.430303


In [178]:
# get labels
query = """SELECT Indicator, Label from codes WHERE Indicator IN (%s, %s);"""
cursor.execute(query, (ind1, ind2) )

x = cursor.fetchall()
print x[0][1]

Land area (sq. km)


In [192]:
p = figure(width=700, height=500, title = 'my title',
	   x_axis_label = df.columns[1], y_axis_label = df.columns[0])
p.circle(x=df[df.columns[1]], y=df[df.columns[0]], size=10,
	 color="navy", alpha=0.5)
p.xaxis.axis_label_text_font_size = "10pt"
p.yaxis.axis_label_text_font_size = "10pt"

In [193]:
# opens in browser
output_file('temp.html')
show(p)